In [1]:
%matplotlib inline

import numpy as np
import h5py

import torch
from torch.nn import functional as F
from torch.autograd import Variable

##############################
from models import deeplab_xception
from models import tiramisu
from models import unet
from datasets import datasets
from utils import manager as mgr
from utils import img_utils
##############################

# set device
#device = torch.device(DEVICE if torch.cuda.is_available() else "cpu")
exp_name = 'unet_df_030_001'
#WEIGHTS_PATH = '/home/philipp/Data/weights/'+exp_name+'/'
WEIGHTS_PATH = '/home/philipp/Code/python/edin_prediction/weights/'
#device = "cuda:0"
device = "cpu"
print(device)

architecture = 'deeplab'#'deeplab'#'densenet'
nr_channels = 7
nr_classes = 4

cpu


In [2]:
## load model with weights

In [3]:
if architecture == 'unet':
    model = unet.UNET(in_channels=nr_channels, out_channels=nr_classes)
    weights_name = 'unet_weights-19-0.217-0.821.pth'
    
elif architecture == 'densenet':
    model = tiramisu.FCDenseNet57(in_channels=nr_channels, n_classes=nr_classes)
    weights_name = 'densenet_weights-3-0.186-0.808.pth'
    
elif architecture == 'deeplab':
    model = deeplab_xception.DeepLabv3_plus(nInputChannels=nr_channels, n_classes=nr_classes, os=16, pretrained=False)
    weights_name = 'deeplab_weights-21-0.192-0.818.pth'
    
else:
    print('provide architecture')
    
try:
    mgr.load_weights(model, WEIGHTS_PATH+weights_name)
    print("weights loaded")
except:
    model.apply(mgr.weights_init)
    print("no weights found")
    
model.to(device)

Constructing DeepLabv3+ model...
Number of classes: 4
Output stride: 16
Number of Input Channels: 7
loading weights '/home/philipp/Code/python/edin_prediction/weights/deeplab_weights-21-0.192-0.818.pth'
loaded weights (lastEpoch 20, loss 0.19151103496551514, error 0.8182618618011475)
weights loaded


DeepLabv3_plus(
  (xception_features): Xception(
    (conv1): Conv2d(7, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (block1): Block(
      (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (rep): Sequential(
        (0): SeparableConv2d_same(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), groups=64, bias=False)
          (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running

In [4]:
## load dataset

In [5]:
class ForestDataset(torch.utils.data.Dataset):

    '''Characterizes a dataset for PyTorch'''

    def __init__(self, path):
        '''Initialization'''
        # open dataset
        self.dset = h5py.File(path, 'r')
        self.ortho = self.dset['ortho']
        self.dsm = self.dset['dsm']
        self.dtm = self.dset['dtm']
        self.slope = self.dset['slope']

        # set number of samples
        self.dataset_size = 10679

        ## TODO:
        # make means and stds load from hdf5
        self.means_tams = np.array([56.12055784563426, 62.130400134006976, 53.03228547781888, 119.50916281232037], dtype='float32')
        self.stds_tams = np.array([30.37628560708646, 30.152693706272483, 23.13718651792004, 49.301477498205074], dtype='float32')

        self.means_dsm = np.array([13.45]).astype(np.float32)
        self.stds_dsm = np.array([10.386633252098674]).astype(np.float32)

        self.means_dtm = np.array([1446.0]).astype(np.float32)
        self.stds_dtm = np.array([271.05322202384195]).astype(np.float32)

        self.means_slope = np.array([22.39]).astype(np.float32)
        self.stds_slope = np.array([11.69830556896441]).astype(np.float32)


    def __len__(self):
        '''Denotes the total number of samples'''
        return self.dataset_size


    def __getitem__(self, index):
        '''Generates one sample of data'''

        # depending on data change mean and std
        means = self.means_tams
        stds = self.stds_tams

        # Load data and get label
        X_ortho = (torch.tensor(self.ortho[index], \
            dtype=torch.float32).permute(2, 0, 1) - \
            means[:, np.newaxis, np.newaxis]) / stds[:, np.newaxis, np.newaxis]
        X_dsm = (torch.tensor(self.dsm[index], \
            dtype=torch.float32).permute(2, 0, 1) - self.means_dsm) / self.stds_dsm
        X_dtm = (torch.tensor(self.dtm[index], \
            dtype=torch.float32).permute(2, 0, 1) - self.means_dtm) / self.stds_dtm
        X_slope = (torch.tensor(self.slope[index], \
            dtype=torch.float32).permute(2, 0, 1) - self.means_slope) / self.stds_slope

        X = torch.cat((X_ortho, X_dsm, X_dtm, X_slope),0)

        return X #torch.from_numpy(y).permute(2, 0, 1)


    def close(self):
        ''' closes the hdf5 file'''
        self.dset.close()


    def show_item(self, index):
        '''shows the data'''
        #plt.imshow(np.array(self.ground_truth[index]))

        fig = plt.figure(figsize=(20,20))

        dic_data = {'RGB' : [np.array(self.ortho[index][:,:,:3]), [0.1, 0.3, 0.5, 0.7]], \
        'CIR' : [np.array(np.roll(self.ortho[index], 1, axis=2)[:,:,:3]), [0.1, 0.3, 0.5, 0.7]], \
        'DSM' : [np.array(self.dsm[index].astype('f')), [10, 20, 30]], \
        'DTM' : [np.array(self.dtm[index].astype('f')), [10, 20, 30]], \
        'Slope' : [np.array(self.slope[index].astype('f')), [10, 20, 30]]}

        for i, key in enumerate(dic_data):
            ax = fig.add_subplot(2, 3, i+1)
            imgplot = plt.imshow(dic_data[key][0])
            ax.set_title(key)
            plt.colorbar(ticks=dic_data[key][1], orientation='horizontal')
            plt.axis('off')

In [6]:
## loading the dataset
#path_dataset = "/home/philipp/Data/dataset_256_df_2.h5"
path_dataset = "/media/philipp/DATA/dataset/dataset_512_df_prediction.h5"
#path_dataset = "/media/philipp/DATA/dataset/dataset_256_df_2.h5"

# open dataset
dataset = ForestDataset(path_dataset)

In [7]:
def quatering(dataset, idx_start, idx_end):
    ## prepare input data
    # cut into 4 quaters = 1 x 7x512x512 -> 4 x 7x256x256
    n = idx_end - idx_start
    store = torch.zeros((n*4,7,256,256),dtype=torch.float32)

    for j,i in enumerate(range(idx_start, idx_end)):
        store[j*4] = dataset[i][:,:256,:256]
        store[j*4+1] = dataset[i][:,256:,:256]
        store[j*4+2] = dataset[i][:,:256,256:]
        store[j*4+3] = dataset[i][:,256:,256:] 
    return store

In [8]:
def predict(store):
    # get predictions
    with torch.no_grad():
        output = model(store)
    pred_256 = mgr.get_predictions(output).numpy()
    return pred_256

In [9]:
def merge(pred_256, idx_start, idx_end):
    ## prepare output data
    # merge into 4 quaters = 4 x 256x256 -> 1 x 512x512
    n = idx_end - idx_start
    pred_512 = np.zeros((n,512,512), dtype=np.int8)
    
    for i in range(n):
        j = i*4
        pred_512[i,:256,:256] = pred_256[j]
        pred_512[i,256:,:256] = pred_256[j+1]
        pred_512[i,:256,256:] = pred_256[j+2]
        pred_512[i,256:,256:] = pred_256[j+3]
    return pred_512    

In [10]:
start = 0
end = 3000
batch = 30

p = np.zeros((end-start,512,512), dtype=np.int8)
print(p.shape)

counter = start

for i in range((end-start) // batch):
    
    store = print(counter, counter+batch)
    
    store = quatering(dataset=dataset, idx_start=counter, idx_end=counter+batch)
    pred_256 = predict(store)
    pred_512 = merge(pred_256, counter, counter+batch)
    
    p[i*batch:(i+1)*batch] = pred_512
    
    counter += batch
    


(3000, 512, 512)
0 30


/home/philipp/miniconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/functional.py:2952: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


30 60
60 90
90 120
120 150
150 180
180 210
210 240
240 270
270 300
300 330
330 360
360 390
390 420
420 450
450 480
480 510
510 540
540 570
570 600
600 630
630 660
660 690
690 720
720 750
750 780
780 810
810 840
840 870
870 900
900 930
930 960
960 990
990 1020
1020 1050
1050 1080
1080 1110
1110 1140
1140 1170
1170 1200
1200 1230
1230 1260
1260 1290
1290 1320
1320 1350
1350 1380
1380 1410
1410 1440
1440 1470
1470 1500
1500 1530
1530 1560
1560 1590
1590 1620
1620 1650
1650 1680
1680 1710
1710 1740
1740 1770
1770 1800
1800 1830
1830 1860
1860 1890
1890 1920
1920 1950
1950 1980
1980 2010
2010 2040
2040 2070
2070 2100
2100 2130
2130 2160
2160 2190
2190 2220
2220 2250
2250 2280
2280 2310
2310 2340
2340 2370
2370 2400
2400 2430
2430 2460
2460 2490
2490 2520
2520 2550
2550 2580
2580 2610
2610 2640
2640 2670
2670 2700
2700 2730
2730 2760
2760 2790
2790 2820
2820 2850
2850 2880
2880 2910
2910 2940
2940 2970
2970 3000


In [11]:
p.shape

(160, 512, 512)

In [12]:
np.save('pred_deeplab.npy', p)

In [ ]:
##################################################################

In [1]:
## prepare input data
# cut into 4 quaters = 1 x 7x512x512 -> 4 x 7x256x256

start = 0
end = 40

n = end - start
store = torch.zeros((n*4,7,256,256),dtype=torch.float32)

for i,j in enumerate(range(0,n*4,4)):
    store[j] = dataset[i][:,:256,:256]
    store[j+1] = dataset[i][:,256:,:256]
    store[j+2] = dataset[i][:,:256,256:]
    store[j+3] = dataset[i][:,256:,256:]

NameError: name 'torch' is not defined

In [10]:
# get predictions
with torch.no_grad():
    output = model(store)
p = mgr.get_predictions(output).numpy()

In [11]:
p.shape

(160, 256, 256)

In [12]:
## prepare output data
# merge into 4 quaters = 4 x 7x256x256 -> 1 x 7x512x512

pred = np.zeros((n,512,512), dtype=np.int8)

for i,j in enumerate(range(0,n*4,4)):
    pred[i,:256,:256] = p[j]
    pred[i,256:,:256] = p[j+1]
    pred[i,:256,256:] = p[j+2]
    pred[i,256:,256:] = p[j+3]

In [13]:
pred.shape

(40, 512, 512)

In [12]:
mean = 119.50916281232037
std = 49.301477498205074

i = 0
l = 10000
nirs = []

for i in range(20):
    # grab data
    x = torch.stack([dataset[i][3], \
                    dataset[i+1*l][3], \
                    dataset[i+2*l][3], \
                    dataset[i+3*l][3]])
    
    # reconstruct tile
    merged_512 = np.ones((512, 512))
    # paste predicted data into array
    merged_512[:256,:256] = dataset[i][3]
    merged_512[256:,:256] = dataset[i+1*l][3]
    merged_512[:256,256:] = dataset[i+2*l][3]
    merged_512[256:,256:] = dataset[i+3*l][3]
    
    merged_512 = std * merged_512 + mean
    
    nirs.append(merged_512)

In [13]:
nirs = np.array(nirs)

In [15]:
np.save('nir.npy', nirs)

In [16]:
np.save('pred.npy', p)

In [ ]:
## convert to geotiff

In [1]:
import os
import numpy as np

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

In [2]:
def array2raster(newRasterfn, dataset, array, dtype):
    """
    save GTiff file from numpy.array
    input:
        newRasterfn: save file name
        dataset : original tif file
        array : numpy.array
        dtype: Byte or Float32.
    """
    cols = array.shape[1]
    rows = array.shape[0]
    originX, pixelWidth, b, originY, d, pixelHeight = dataset.GetGeoTransform() 

    driver = gdal.GetDriverByName('GTiff')

    # set data type to save.
    GDT_dtype = gdal.GDT_Unknown
    if dtype == "Byte": 
        GDT_dtype = gdal.GDT_Byte
    elif dtype == "Float32":
        GDT_dtype = gdal.GDT_Float32
    else:
        print("Not supported data type.")

    # set number of band.
    if array.ndim == 2:
        band_num = 1
    else:
        band_num = array.shape[2]

    outRaster = driver.Create(newRasterfn, cols, rows, band_num, GDT_dtype)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))

    # Loop over all bands.
    for b in range(band_num):
        outband = outRaster.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        if band_num == 1:
            outband.WriteArray(array)
        else:
            outband.WriteArray(array[:,:,b])

    # setteing srs from input tif file.
    prj=dataset.GetProjection()
    outRasterSRS = osr.SpatialReference(wkt=prj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [3]:
pred = np.load('pred_deeplab.npy')

In [4]:
def find_files(dir_img, sort=False):
    """
    find paths for provided data types
    inputs:
        dir_img (str) : directory path
    return:
        paths (dictionary) : dictionary containing file paths for each of the data types
    """

    paths = []
    # loop over all files found in directory and retrive indices
    for file in os.listdir("{}".format(dir_img)):
        if file[-4:] == ".tif":
            paths.append(dir_img + file)

    if sort:
        paths = sorted(paths)
    
    return paths

In [5]:
path_input = find_files('/media/philipp/DATA/2018_tamsweg/ortho/', sort=True)

In [6]:
path_input[0:3]

['/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_121889.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_121890.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122277.tif']

In [7]:
len(path_input)

10679

In [8]:
pred.shape[0]

160

In [9]:
# convert numpy array to geotiff files
for i, path_in in enumerate(path_input[:pred.shape[0]]):
    path_out = 'foto/pred_{}.tif'.format(i)
    dataset = gdal.Open(path_in, gdal.GA_ReadOnly)
    array2raster(newRasterfn=path_out, dataset=dataset, array=pred[i], dtype='Byte')

In [10]:
# mosaik geotiff files

def mosaik_raster(input_files, output_file):
    
    # set path to text file with all tif file paths to be merged
    merge_list_path =  '/home/philipp/Code/python/edin_prediction/dtm_merge_list.txt'
    # create txt file with all tif files needed
    with open(merge_list_path, 'w+') as f:
        for name in input_files:
            f.write(name + '\n')

    # create string containing bash command
    cmd = "gdal_merge.py -ot Byte -of GTiff -o {} --optfile {}".format(output_file, merge_list_path)

    # execute bash command
    os.system(cmd)

    print("DTM file created - files merged")

In [11]:
input_files = find_files('/home/philipp/Code/python/edin_prediction/foto/', sort=True)

In [12]:
output_file = '/home/philipp/Code/python/edin_prediction/pred/pred.tif'
mosaik_raster(input_files, output_file)

DTM file created - files merged


In [13]:
vector_file = '/home/philipp/Code/python/edin_prediction/pred/pred.shp'
# create vector (polygons) out of raster
cmd = 'gdal_polygonize.py -8 {} -b 1 -f "ESRI Shapefile" {} output DF'.format(output_file, vector_file)
# execute bash command
os.system(cmd)

0

In [26]:
42/160*10679

2803.2375